In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers, optimizers, datasets
import numpy as np

In [2]:
x = tf.random.normal(shape=(128,28,28,1), mean=0., stddev=1.) # [batch size, W, H, channel]
x.shape

TensorShape([128, 28, 28, 1])

### layers.Conv2d

* strides=1

In [3]:
x_after_conv1 = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) # filters=32 : Channel의 갯수가 32개
print('x_after_conv1 :',x_after_conv1.shape)

x_after_conv1 : (128, 26, 26, 32)


* strides=2

In [4]:
x_after_conv2 = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(x)
print('x_after_conv2 :',x_after_conv2.shape)

x_after_conv2 : (128, 13, 13, 32)


In [5]:
x_after_conv3 = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation="relu")(x) 
# padding='same' : input과 output이 같은 크기를 갖도록 함
print('x_after_conv3 :',x_after_conv3.shape)

x_after_conv3 : (128, 28, 28, 32)


### Max Pooling

In [6]:
x = tf.random.normal(shape=(128,28,28,1), mean=0., stddev=1.) # [batch size, W, H, channel]
x.shape

TensorShape([128, 28, 28, 1])

In [7]:
x_after_maxpool1 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)
print('x_after_maxpool1:',x_after_maxpool1.shape)

x_after_maxpool1: (128, 14, 14, 1)


* pool_size=(4,4), padding='same'

In [8]:
x_after_maxpool2 = layers.MaxPooling2D(pool_size=(4,4), padding='same', strides=(4,4))(x)
print('x_after_maxpool2:',x_after_maxpool2.shape)

x_after_maxpool2: (128, 7, 7, 1)


### Average Pooling

In [9]:
x = tf.random.normal(shape=(128,28,28,1), mean=0., stddev=1.) # [batch size, W, H, channel]
x.shape

TensorShape([128, 28, 28, 1])

In [10]:
x_after_avgpool1 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2))(x)
print('x_after_avgpool1:',x_after_avgpool1.shape)

x_after_avgpool1: (128, 14, 14, 1)


In [11]:
x_after_avgpool2 = layers.AveragePooling2D(pool_size=(7,7), strides=(7,7))(x)
print('x_after_avgpool2:',x_after_avgpool2.shape)

x_after_avgpool2: (128, 4, 4, 1)


### Global Average Pooling

In [12]:
x = tf.random.normal(shape=(128,28,28,1), mean=0., stddev=1.) # [batch size, W, H, channel]
x.shape

TensorShape([128, 28, 28, 1])

* GlobalAveragePooling2D() : input값에 아무것도 없어도 1로

In [13]:
x_after_global_avgpool1 = layers.GlobalAveragePooling2D()(x)
print('x_after_global_avgpool1:',x_after_global_avgpool1.shape)

x_after_global_avgpool1: (128, 1)


## CNN application

In [14]:
(images_train,labels_train), (images_test,labels_test) = datasets.mnist.load_data()
print('----------------train----------------')
print('images:',images_train.shape, images_train.min(), images_train.max())
print('labels:',labels_train.shape)
images_train = images_train.reshape((60000,28,28,1)) # reshape
images_train = images_train.astype("float32")/255 # 0~1
print('normalized images:',images_train.shape, images_train.min(), images_train.max())

print('----------------test----------------')
print('images:',images_test.shape, images_test.min(), images_test.max())
print('labels:',labels_test.shape)
images_test = images_test.reshape((10000,28,28,1))
images_test = images_test.astype("float32")/255
print('normalized images:',images_test.shape, images_test.min(), images_test.max())

----------------train----------------
images: (60000, 28, 28) 0 255
labels: (60000,)
normalized images: (60000, 28, 28, 1) 0.0 1.0
----------------test----------------
images: (10000, 28, 28) 0 255
labels: (10000,)
normalized images: (10000, 28, 28, 1) 0.0 1.0


In [15]:
input_shape = (28, 28, 1)
num_classes = 10
model = tf.keras.models.Sequential([
    # kernel : (3,3)
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPool2D(strides=(2,2)),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [16]:
model_train = model.fit(images_train, labels_train,
                    batch_size=128,
                    epochs=3,
                    validation_split=0.2)

Epoch 1/3
375/375 [==============================] - 42s 108ms/step - loss: 0.2250 - acc: 0.9324 - val_loss: 0.0693 - val_acc: 0.9784
Epoch 2/3
375/375 [==============================] - 40s 106ms/step - loss: 0.0568 - acc: 0.9826 - val_loss: 0.0501 - val_acc: 0.9858
Epoch 3/3
375/375 [==============================] - 39s 104ms/step - loss: 0.0369 - acc: 0.9884 - val_loss: 0.0386 - val_acc: 0.9877


In [17]:
test_loss, test_acc = model.evaluate(images_test,  labels_test, verbose=2)

print('test accuracy:', test_acc)

313/313 - 2s - loss: 0.0321 - acc: 0.9888 - 2s/epoch - 6ms/step
test accuracy: 0.9887999892234802


### 모델 저장

In [18]:
import os

* 모델 저장 경로

In [19]:
model_save_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./cnn_model.ckpt",
                                                         save_weights_only=True,
                                                         verbose=2)

In [20]:
model_train = model.fit(images_train, labels_train,
                        batch_size=128,
                        epochs=3,
                        validation_split=0.2,
                        callbacks=[model_save_callback])

Epoch 1/3
375/375 [==============================] - ETA: 0s - loss: 0.0288 - acc: 0.9910
Epoch 1: saving model to .\cnn_model.ckpt
375/375 [==============================] - 39s 104ms/step - loss: 0.0288 - acc: 0.9910 - val_loss: 0.0366 - val_acc: 0.9883
Epoch 2/3
375/375 [==============================] - ETA: 0s - loss: 0.0202 - acc: 0.9937
Epoch 2: saving model to .\cnn_model.ckpt
375/375 [==============================] - 41s 109ms/step - loss: 0.0202 - acc: 0.9937 - val_loss: 0.0370 - val_acc: 0.9887
Epoch 3/3
375/375 [==============================] - ETA: 0s - loss: 0.0166 - acc: 0.9948
Epoch 3: saving model to .\cnn_model.ckpt
375/375 [==============================] - 41s 111ms/step - loss: 0.0166 - acc: 0.9948 - val_loss: 0.0361 - val_acc: 0.9898


### 모델 불러오기

In [22]:
checkpoint_path="./cnn_model.ckpt"
model.load_weights(checkpoint_path)

In [23]:
loss,acc = model.evaluate(images_test,  labels_test, verbose=2)
print("accuracy: {:5.2f}%".format(100*acc))

313/313 - 5s - loss: 0.0260 - acc: 0.9904 - 5s/epoch - 15ms/step
accuracy: 99.04%
